## Import modular

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Parameters



In [ ]:
# Set random seed for reproducibility
np.random.seed(2025)

# Parameters
num_beers = 20  # 20 types of beers
initial_price_per_ml = 150 / 500  # Price per ml (0.3 NTD/ml)
operation_hours = 13  # 11:00 to 00:00 (13 hours)
time_intervals = 26  # 30-minute intervals (13*2)
price_increase_rate = 0.005  # 0.5% price increase per 500 ml sold
price_decrease_rate = 0.04  # 4% price decrease if no sales
upper_limit = 1.2  # 120% of initial price
lower_limit = 0.5  # 50% of initial price
num_simulations = 1000  # Number of simulations for a week
ml_per_unit = 500  # Price adjustment threshold (500 ml)

# Classify beers into popular and non-popular ####不要是50 50
popular_beers = np.random.choice(num_beers, size=num_beers // 2, replace=False)
non_popular_beers = np.setdiff1d(np.arange(num_beers), popular_beers)

# Customer types
customer_types = ["loyal", "price_high", "price_low"]

# Time slot definitions
time_slots = {
    'lunch': list(range(0, 6)),  # 11:00-14:00
    'tea': list(range(6, 12)),   # 14:00-17:00
    'dinner': list(range(12, 18)),  # 17:00-20:00
    'night': list(range(18, 26))  # 20:00-24:00
}

# Time multipliers for demand
time_slot_multipliers = {
    'lunch': 1.2,
    'tea': 0.8,
    'dinner': 1.5,
    'night': 1.8
}

## Simulate customer type

In [ ]:
def simulate_customer_type():
    return np.random.choice(customer_types, p=[0.4, 0.3, 0.3])

## Simulate demand

In [ ]:
def get_current_time_slot(interval):
    for slot, intervals in time_slots.items():
        if interval in intervals:
            return slot
    return 'night'

def simulate_demand(interval, day):
    base_demand = np.random.randint(5, 80)  # 提高最低需求，避免完全無顧客

    busy_days = [5, 6, np.random.randint(0, 5)]  # 週末或隨機天為繁忙日
    day_multiplier = 1.5 if day in busy_days else 1.0

    time_slot = get_current_time_slot(interval)
    time_multiplier = time_slot_multipliers[time_slot]

    # 讓需求帶有波動但不至於過低
    noise = np.random.normal(1, 0.2)  # 加入波動噪聲

    return max(1, int(base_demand * day_multiplier * time_multiplier * noise))  # 保證最低一位顧客

## Simulate volume

In [ ]:
def simulate_volume():
    return np.random.randint(250, 1000)  # Each customer buys 250-1000 ml

## Simulate beer choice based on type and price

In [ ]:
def simulate_beer_choice(prices, customer_type):
    if customer_type == "loyal":
        return np.random.choice(popular_beers)
    elif customer_type == "price_high":
        probabilities = np.exp(-prices * 0.1) / np.sum(np.exp(-prices * 0.1))  # 減少高價懲罰
    else:
        probabilities = np.exp(-prices * 0.4) / np.sum(np.exp(-prices * 0.4))  # 降低對價格低的偏好

    return np.random.choice(np.arange(num_beers), p=probabilities)

def update_popularity(total_sales_volume):
    global popular_beers, non_popular_beers

    # 如果總銷量為 0，均勻分配概率
    if total_sales_volume.sum() == 0:
        total_sales_probabilities = np.full(num_beers, 1 / num_beers)
    else:
        # Normalize sales volume to probabilities
        total_sales_probabilities = total_sales_volume / total_sales_volume.sum()

    # Select 5 popular beers based on probabilities
    popular_beers = np.random.choice(
        np.arange(num_beers),
        size=5,
        replace=False,
        p=total_sales_probabilities
    )

    # Update non-popular beers
    non_popular_beers = np.setdiff1d(np.arange(num_beers), popular_beers)

## Simulation function

In [ ]:
def run_simulation(dynamic_pricing=True):
    total_revenue_by_day_and_timeslot = np.zeros((7, 26))  # 7 days, 26 intervals (13*2)
    total_revenue = 0
    beer_prices = np.full(num_beers, initial_price_per_ml)
    price_history = []
    total_sales_volume = np.zeros(num_beers)

    for day in range(7):
        # 每週開始前重設熱門啤酒
        if day == 0:
            update_popularity(total_sales_volume)

        daily_sales = np.zeros(num_beers)  # 初始化 daily_sales 為 0

        for interval in range(time_intervals):
            total_sales = np.zeros(num_beers)
            num_customers = simulate_demand(interval, day)

            for _ in range(num_customers):
                customer_type = simulate_customer_type()
                beer_idx = simulate_beer_choice(beer_prices, customer_type)
                volume = simulate_volume()

                total_sales[beer_idx] += volume
                total_sales_volume[beer_idx] += volume
                total_revenue += volume * beer_prices[beer_idx]

                total_revenue_by_day_and_timeslot[day, interval] += volume * beer_prices[beer_idx]

            # 每30分鐘即時更新價格
            if dynamic_pricing:
                for i in range(num_beers):
                    if total_sales[i] >= ml_per_unit:
                        beer_prices[i] *= (1 + price_increase_rate * (total_sales[i] / ml_per_unit))
                    elif total_sales[i] == 0:
                        beer_prices[i] *= (1 - price_decrease_rate)  # 完全無銷售才降價
                    else:
                        beer_prices[i] *= 1  # 有銷售但未達標，價格維持不變

                    # 漲停與跌停控制
                    if beer_prices[i] >= initial_price_per_ml * upper_limit:
                        beer_prices[i] = initial_price_per_ml * upper_limit  # 漲停價
                    elif beer_prices[i] <= initial_price_per_ml * lower_limit:
                        beer_prices[i] = initial_price_per_ml * lower_limit  # 跌停價

            daily_sales += total_sales
            price_history.append(beer_prices.copy())  # 記錄價格變動

    return total_revenue, np.array(price_history), total_sales_volume, total_revenue_by_day_and_timeslot

## Run simulations for dynamic and static pricing


In [ ]:
# Run simulations for dynamic and static pricing
dynamic_results = [run_simulation(dynamic_pricing=True) for _ in range(num_simulations)]
static_results = [run_simulation(dynamic_pricing=False) for _ in range(num_simulations)]

# Extract revenue, price history, and sales volume
dynamic_revenue = [result[0] for result in dynamic_results]
static_revenue = [result[0] for result in static_results]
price_history = dynamic_results[0][1]  # Take price history from the first simulation
total_sales_volume = np.mean([result[2] for result in dynamic_results], axis=0)

# Calculate average revenue
avg_dynamic_revenue = np.mean(dynamic_revenue)
avg_static_revenue = np.mean(static_revenue)

## 每個時段的比較圖

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Time slots labels
time_slots_labels = ['lunch', 'tea', 'dinner', 'night']

# Define the days of the week
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

def aggregate_revenue_by_timeslot(total_revenue_by_day_and_timeslot):
    """
    Aggregates the revenue data by time slots for each day.
    """
    revenue_by_day_and_timeslot = np.zeros((7, 4))  # 7 days, 4 time slots

    # Define time slot boundaries based on intervals (13 hours * 2 intervals)
    time_slot_indices = {
        'lunch': list(range(0, 6)),  # 11:00-14:00
        'tea': list(range(6, 12)),   # 14:00-17:00
        'dinner': list(range(12, 18)),  # 17:00-20:00
        'night': list(range(18, 26))  # 20:00-24:00
    }

    for day_idx in range(7):  # Loop through each day
        for time_slot, indices in time_slot_indices.items():
            time_slot_idx = time_slots_labels.index(time_slot)
            revenue_by_day_and_timeslot[day_idx, time_slot_idx] = np.sum(total_revenue_by_day_and_timeslot[day_idx, indices])

    return revenue_by_day_and_timeslot

# Extract the total revenue per day and per time slot for each simulation
dynamic_revenue_by_day_and_timeslot = np.array([result[3] for result in dynamic_results])  # result[3] contains total_revenue_by_day_and_timeslot
static_revenue_by_day_and_timeslot = np.array([result[3] for result in static_results])

# Aggregate the revenue by time slots
dynamic_revenue_aggregated = np.mean([aggregate_revenue_by_timeslot(result) for result in dynamic_revenue_by_day_and_timeslot], axis=0)
static_revenue_aggregated = np.mean([aggregate_revenue_by_timeslot(result) for result in static_revenue_by_day_and_timeslot], axis=0)

# Create the plots
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot for Dynamic Pricing
for i, day in enumerate(days_of_week):
    axes[0].plot(time_slots_labels, dynamic_revenue_aggregated[i], label=day)
axes[0].set_title("Dynamic Pricing Strategy")
axes[0].set_xlabel("Time Slots")
axes[0].set_ylabel("Average Revenue (NTD)")
axes[0].legend(title="Days")

# Plot for Static Pricing
for i, day in enumerate(days_of_week):
    axes[1].plot(time_slots_labels, static_revenue_aggregated[i], label=day)
axes[1].set_title("Static Pricing Strategy")
axes[1].set_xlabel("Time Slots")
axes[1].set_ylabel("Average Revenue (NTD)")
axes[1].legend(title="Days")

# Display the plots
plt.tight_layout()
plt.show()

## Plot comparison

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(dynamic_revenue, bins=50, alpha=0.6, label='Dynamic Pricing')
plt.hist(static_revenue, bins=50, alpha=0.6, label='Static Pricing')
plt.axvline(avg_dynamic_revenue, color='blue', linestyle='dashed', linewidth=2, label=f'Avg Dynamic: {avg_dynamic_revenue:.2f}')
plt.axvline(avg_static_revenue, color='orange', linestyle='dashed', linewidth=2, label=f'Avg Static: {avg_static_revenue:.2f}')
plt.xlabel('Total Revenue (NTD)')
plt.ylabel('Frequency')
plt.title('Revenue Comparison: Dynamic vs Static Pricing (10000 Simulations)')
plt.legend()
plt.show()

## Plot price changes for each beer

In [ ]:
plt.figure(figsize=(12, 8))
for i in range(num_beers):
    plt.plot(price_history[:, i], label=f'Beer {i + 1}')
plt.xlabel('Slots')
plt.ylabel('Price (NTD)')
plt.title('Price Changes of Beers Over 7 Days (Dynamic Pricing)')
plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1))
plt.show()

## Plot total sales volume per beer

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(range(1, num_beers + 1), total_sales_volume)
plt.xlabel('Beer Type')
plt.ylabel('Total Volume Sold (ml)')
plt.title('Total Sales Volume for Each Beer (Dynamic Pricing)')
plt.show()

In [ ]:
print(f'Average Dynamic Pricing Revenue: {avg_dynamic_revenue:.2f} NTD')
print(f'Average Static Pricing Revenue: {avg_static_revenue:.2f} NTD')
print('Total Sales Volume (ml) per Beer:', total_sales_volume)

1. 啤酒種類和定價
	•	假設有 20 種不同類型的啤酒，每種啤酒的價格設定為每毫升 0.3 NTD。
	•	初始定價為每毫升 0.3 NTD，並設定了價格調整的閾值和限值（價格上限為 120% 的初始價格，下限為 50% 的初始價格）。

2. 營業時間和時間區間
	•	假設營業時間為 13 小時（從 11:00 到 00:00），並且一天分為 26 個 30 分鐘的時間區間（即 13 小時×2）。
	•	每個時間區間對應不同的需求乘數，例如午餐時間需求會增加 20%、晚餐時間需求會增加 50%。

3. 客戶類型與需求
	•	假設有三種類型的客戶：忠實顧客、價格敏感顧客（高價）和價格敏感顧客（低價）。
	•	客戶在不同的時間區間和日子（平日或週末）會有不同的需求。需求量會根據顧客類型和時間區間（如午餐、晚餐、夜間等）進行調整。

4. 銷售量與定價策略
	•	假設每個顧客會購買 250 到 400 毫升的啤酒，並且啤酒的選擇會根據顧客類型來決定。忠實顧客偏好熱門啤酒，而價格敏感顧客則根據啤酒的價格高低做出選擇。
	•	動態定價：根據銷售量調整價格。如果某種啤酒的銷量超過設定的閾值（500 毫升），則提高價格；如果銷量不足，則降低價格，並且價格必須在設定的上下限範圍內。
	•	靜態定價：所有啤酒的價格始終保持不變，維持在初始價格。

5. 模擬結果
	•	模擬運行 10,000 次，分別針對動態定價和靜態定價策略進行，並跟踪每個模擬中的總收入、啤酒價格歷史和總銷售量。
	•	最終，腳本計算並比較了動態定價和靜態定價的平均收入。

6. 結果分析
	•	通過這些假設，模擬結果將顯示動態定價是否能夠帶來比靜態定價更高的總收入，並分析價格調整如何影響啤酒銷售的總量。